<a href="https://colab.research.google.com/github/Rafsan7238/BracU_Thesis_P2/blob/main/Pre_Thesis_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Application of Deep Convolutional Neural Network in Breast Cancer Prediction Using Digital Mammograms**

**Authors:** Rafsan Al Mamun, Md. Al Imran Sefat, Gazi Abu Rafin, Adnan

**Project GitHub Link:** [https://github.com/Rafsan7238/BracU_Thesis_P2](https://github.com/Rafsan7238/BracU_Thesis_P2)



---



***Abstract:*** *Cancer, a diagnosis that is so dreaded and scary, that its fear alone can strike even the
strongest of souls. The disease is often thought of as untreatable and unbearably painful, with
usually, no cure available. Among all the cancers, breast cancer is the second most deadliest ,
especially among women. What decides the patients’ fate is the early diagnosis of the cancer,
facilitating subsequent clinical management. Mammography plays a vital role in the
screening of breast cancers as it can detect any breast masses or calcifications early. However,
the extremely dense breast tissues pose difficulty in the detection of cancer mass, thus,
encouraging the use of machine learning (ML) techniques and artificial neural networks
(ANN) to assist radiologists in faster cancer diagnosis. This paper explores the MIAS
database, containing 332 digital mammograms from women, which were augmented and
preprocessed, and fed into different convolutional neural network (CNN) models, with the
aim of differentiating healthy tissues from cancerous ones with high accuracy. The paper,
along with a new proposed CNN model for better identification of breast cancer, focuses on
the significance of computer-aided detection (CAD) models overall in the early diagnosis of
breast cancer. While a diagnosis of breast cancer may still leave patients dreaded, we believe
our research can be a symbol of hope for all.*

## **Data Collection and Preprocessing**

The dataset used for this project has been collected from the MIAS dataset. The dataset consisted of raw breast mammograms of 161 patients having healthy, malignant or benign breast tissues. 

These raw mammograms have been divided into 2 groups: healthy and cancer, having mammograms of patients with healthy and benign tissues, and malignant tissues respectively. 

However, the dataset is too small to be used in CNNs. Hence, they had to be augmented using the following processes in order to increase the no. of mammograms to be worked with. 

In [18]:
import cv2
from google.colab.patches import cv2_imshow
import os

def load_data():
  """
    Load image data from directory '/content/BracU_Thesis_P2/Dataset'.

    Load each image file from the subdirectories of Dataset, turn it into B/W and augment it. 
    After each augmentation append the images into the images list, and their corresponding
    labels in the labels list. 

    Return tuple `(images, labels)`. `images` should be a list of all
    of the images in the data directory, where each image is formatted as a
    numpy ndarray with dimensions 1024 x 1024 x 1. `labels` should
    be a list of labels (healthy or cancer), representing the categories for each of the
    corresponding `images`.
    """

  images = []
  labels = []

  # Loop through the healthy dataset
  directory_path = "/content/drive/MyDrive/Thesis/Dataset/Healthy"
  os.chdir(directory_path)

  for file in os.listdir():
    if file.endswith(".jpg"):
      print(f"Working with file: {file}")

      img = cv2.imread(directory_path + "/" + file)

      #TODO

      images.append(img)
      labels.append("healthy")


  # Same for cancer dataset


  # Return a tuple of (images, labels)

  return (images, labels)


# main_method to check function
image, label = load_data()
cv2_imshow(image[0])
print(label[0])

Working with file: mdb017.jpg
Working with file: mdb007.jpg
Working with file: mdb006.jpg
Working with file: mdb016.jpg
Working with file: mdb002.jpg
Working with file: mdb014.jpg
Working with file: mdb013.jpg
Working with file: mdb012.jpg
Working with file: mdb003.jpg
Working with file: mdb004.jpg
Working with file: mdb008.jpg
Working with file: mdb015.jpg
Working with file: mdb001.jpg
Working with file: mdb009.jpg
Working with file: mdb005.jpg
Working with file: mdb011.jpg
Working with file: mdb010.jpg
Working with file: mdb029.jpg
Working with file: mdb047.jpg
Working with file: mdb040.jpg
Working with file: mdb037.jpg
Working with file: mdb025.jpg
Working with file: mdb052.jpg
Working with file: mdb043.jpg
Working with file: mdb041.jpg
Working with file: mdb048.jpg
Working with file: mdb051.jpg
Working with file: mdb033.jpg
Working with file: mdb031.jpg
Working with file: mdb021.jpg
Working with file: mdb044.jpg
Working with file: mdb019.jpg
Working with file: mdb042.jpg
Working wi

TypeError: ignored